In [1]:
import os
import sys

import numpy as np
from math import sqrt
from collections import defaultdict

sys.path.insert(0,os.path.abspath('../'))
from FP_Utilities import Read_E_Bins, Build_Nagy_Weighted_FPs, Build_Spline_Weighted_FPs, Build_GEF_Weighted_FPs
from FP_Utilities import Read_Fission_Spectrum, writeout

rootdir=os.path.abspath(os.getcwd())

In [2]:
# Determine fission fraction by isotope
# From ETA Ouput - Not explicitly required for program
tot=1.23846E-04*.85+2.42397E-04*.15
tot_234=9.32577E-07*.85+2.16850E-06*.15
tot_235=1.21540E-04*.85+2.33367E-04*.15
tot_236=1.27836E-07*.85+4.97452E-07*.15
tot_238=1.17277E-06*.85+6.27153E-06*.15
wf_234=0.0093898
wf_235=0.93217
wf_236=0.003
wf_238=0.05544
print "235",tot_235/tot
print "238",(tot_234+tot_236+tot_238)/tot   # 234 and 236 lumped with 238 because minor and threshold fissioners


235 0.976596543143
238 0.0228684510514


In [6]:
# User Inputs
#---------------------------------------------------------------------------------------#  
fiss235Name = 'Data/E_fiss_235.csv'   # Fission spectra input file
fiss238Name = 'Data/E_fiss_238.csv'   # Fission spectra input file
binsName = 'Data/Bins.csv'   # Bin structure input file
fp235Name = '../235_data.xlsx'  # 235 energy dependent data - must contain same isotopes as 238
fp238Name = '../238_data.xlsx'  # 238 energy dependent data - must contain same isotopes as 235
gef235Dir = '../../Validation/235Watt' # Path to GEF data
gef238Dir = '../../Validation/238Watt' # Path to GEF data

f235=0.976596543143   # fraction of fissions caused by 235 - f_235+f_238 should equal 1
f238=0.0228684510514   # fraction of fissions caused by 238 - f_235+f_238 should equal 1
#---------------------------------------------------------------------------------------#  

In [7]:
### Nagy Based Predictions ###

# Read in the energy bin structure
(lower_bins,upper_bins,bins)=Read_E_Bins(binsName)

# Calculate 235 data
(fiss_235_e,fiss_235)=Read_Fission_Spectrum(fiss235Name)
y_235, err_235=Build_Nagy_Weighted_FPs(fp235Name,bins,f235,fiss_235)

# Calculate 235 data
(fiss_238_e,fiss_238)=Read_Fission_Spectrum(fiss238Name)
y_238, err_238=Build_Nagy_Weighted_FPs(fp238Name,bins,f238,fiss_238)

# Combine the data into a single data set
y = {}
err = {}
absErr = {}
out = []
for A in y_235.keys():
    y[A]=y_235[A]+y_238[A]
    err[A]=sqrt(err_235[A]**2+err_238[A]**2)
    absErr[A]=err[A]*y[A]
    out.append((A,y[A],absErr[A]))

# Output Results
print y, absErr
writeout("Obj_Nagy_fy.csv",sorted(out,key=lambda l:l[0]))

{97: 5.7461782160369692, 99: 5.659760169429747, 133: 6.4055467023653065, 161: 0.0010419855309903399, 140: 5.7096104269085428, 141: 5.4916051796477952, 111: 0.24759108771408453, 115: 0.24533858566399672, 113: 0.17557645484317863, 147: 2.1213136682130789, 151: 0.47052504885334068, 153: 0.17581713693361517, 156: 0.026316141923971414, 95: 6.1714813146593848} {97: 0.089756060016041, 99: 0.089434558491184216, 133: 0.13418492265333443, 161: 4.3194287596086492e-05, 140: 0.067133128663608474, 141: 0.097657069083106243, 111: 0.0068063467583643327, 115: 0.011801091543391798, 113: 0.011183170972419988, 147: 0.032216131380808956, 151: 0.016756704014549064, 153: 0.0073856730025406592, 156: 0.00090074938315589932, 95: 0.089146528623917476}


In [8]:
### Spline Based Predictions ###

# Read in the energy bin structure
(lower_bins,upper_bins,bins)=Read_E_Bins(binsName)

# Calculate 235 data
(fiss_235_e,fiss_235)=Read_Fission_Spectrum(fiss235Name)
y_235=Build_Spline_Weighted_FPs(fp235Name,bins,f235,fiss_235)

# Calculate 235 data
(fiss_238_e,fiss_238)=Read_Fission_Spectrum(fiss238Name)
y_238=Build_Spline_Weighted_FPs(fp238Name,bins,f238,fiss_238)

# Combine the data into a single data set
y=defaultdict(list)
out=[]
for A in y_235.keys():
    y[A]=np.sum(np.asarray(y_235[A])+np.asarray(y_238[A]))
    out.append((A,y[A],0.0))
print y

# Output Results
writeout("Obj_Spline_fy.csv",sorted(out,key=lambda l:l[0]))

defaultdict(<type 'list'>, {97: 5.7204751017269366, 99: 5.5331969363014224, 133: 6.3925519499271095, 161: 0.0011423538319074945, 140: 5.6859873140078472, 141: 5.7718468237012504, 111: 0.24601976814242696, 115: 0.23344208327219276, 113: 0.19376145548017212, 147: 2.1694131853375165, 151: 0.43845254918343524, 153: 0.17797918154000195, 156: 0.029069439139942616, 95: 6.2102475104570551})


In [9]:
### GEF Based Predictions ###

# Read in the energy bin structure
(lower_bins,upper_bins,bins)=Read_E_Bins(binsName)

# Calculate 235 data
y_235={}
err_235={}
(y_235,err_235)=Build_GEF_Weighted_FPs(bins,f235*fiss_235,gef235Dir)

# Calculate 238 data
y_238={}
err_238={}
(y_238,err_238)=Build_GEF_Weighted_FPs(bins,f238*fiss_238,gef238Dir)

# Combine data into a single data set
y={}
err={}
absErr={}
out=[]
for A in y_235.keys():
    try:
        y[A]=y_235[A]+y_238[A]
        err[A]=sqrt(err_235[A]**2+err_238[A]**2)
        absErr[A]=err[A]*y[A]
        out.append((A,y[A],err[A],absErr[A]))
    except KeyError as e:
        print "Key error: {0} not found for both isotopes.".format(A)

# Output Results
writeout("Obj_GEF_fy.csv",sorted(out,key=lambda l:l[0]))





47 46 46
Key error: 58 not found for both isotopes.
Key error: 54 not found for both isotopes.
Key error: 52 not found for both isotopes.
Key error: 180 not found for both isotopes.
Key error: 181 not found for both isotopes.
